In [ ]:
####################################
#ENVIRONMENT SETUP

In [ ]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr

import sys; import os; import time; from datetime import timedelta
import pickle
import h5py
from tqdm import tqdm

In [ ]:
#MAIN DIRECTORIES
def GetDirectories():
    mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
    mainCodeDirectory=os.path.join(mainDirectory,"Code/CodeFiles/")
    scratchDirectory='/mnt/lustre/koa/scratch/air673/'
    codeDirectory=os.getcwd()
    return mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory

[mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory] = GetDirectories()

In [ ]:
#IMPORT CLASSES
sys.path.append(os.path.join(mainCodeDirectory,"2_Variable_Calculation"))
from CLASSES_Variable_Calculation import ModelData_Class, SlurmJobArray_Class, DataManager_Class

In [ ]:
#IMPORT FUNCTIONS
sys.path.append(os.path.join(mainCodeDirectory,"2_Variable_Calculation"))
import FUNCTIONS_Variable_Calculation
from FUNCTIONS_Variable_Calculation import * # import NumericalFunctions 

In [ ]:
####################################
#LOADING CLASSES

In [ ]:
#data loading class
ModelData = ModelData_Class(mainDirectory, scratchDirectory, simulationNumber=2)
#data manager class
DataManager = DataManager_Class(mainDirectory, scratchDirectory, ModelData.res, ModelData.t_res, ModelData.Nz_str,
                                ModelData.Np_str, dataType="LagrangianArrays", dataName="BUDGET_VARS",
                                dtype='float32')

In [ ]:
#JOB ARRAY SETUP
UsingJobArray=True

def GetNumJobs(res,t_res):
    if res=='1km':
        if t_res=='5min':
            num_jobs=20
        elif t_res=='1min':
            num_jobs=100
    elif res=='250m': 
        if t_res=='1min':
            num_jobs=500
    return num_jobs
num_jobs = GetNumJobs(ModelData.res,ModelData.t_res)
SlurmJobArray = SlurmJobArray_Class(total_elements=ModelData.Ntime, num_jobs=num_jobs, UsingJobArray=UsingJobArray)
start_job = SlurmJobArray.start_job; end_job = SlurmJobArray.end_job

def GetNumElements():
    loop_elements = np.arange(ModelData.Ntime)[start_job:end_job]
    return loop_elements
loop_elements = GetNumElements()

In [ ]:
####################################
#FUNCTIONS

In [ ]:
def GetZYX(inputParcelDirectory, timeString):
    #Lagrangian Position Arrays
    ##############
    def grid_location(z,y,x):
        zf=ModelData.zf*1000; which_zh=np.clip(np.searchsorted(zf,z)-1,0,None).astype(np.uint16)
        #which_zh=np.where(which_zh == -1, 0, which_zh) 
        
        yf=ModelData.yf*1000; which_yh=np.clip(np.searchsorted(yf,y)-1,0,None).astype(np.uint16) 
        #which_yh=np.where(which_yh == -1, 0, which_yh) 
        
        xf=ModelData.xf*1000; which_xh=np.clip(np.searchsorted(xf,x)-1,0,None).astype(np.uint16)
        #which_xh=np.where(which_xh == -1, 0, which_xh) 
        
        return which_zh,which_yh,which_xh
    
    # print('Creating Lagrangian X,Y,Z Binary Arrays')
    x = DataManager.GetTimestepParcel(DataManager.inputParcelDirectory, timeString, variableName="x")
    y = DataManager.GetTimestepParcel(DataManager.inputParcelDirectory, timeString, variableName="y")
    z = DataManager.GetTimestepParcel(DataManager.inputParcelDirectory, timeString, variableName="z")
    Z,Y,X=grid_location(z,y,x)
    return z,x,Z,Y,X

In [ ]:
def MakeLagrangianArray(inputDataDirectory, timeString,varnames,Z,Y,X): #FAST ADVANCED INDEXING VERSION
    # print('Making Variables Lagrangian Array')
    # Initialize dictionaries
    var_data_dict = {varname: CallVariable(ModelData, DataManager, timeString, varname) for varname in varnames}
    VAR = {varname: np.zeros_like(Z, dtype='float32') for varname in varnames}

    # print('Running')
    # Nt = len(data['time'])
    # Get Indices
    z_idx = Z
    y_idx = Y
    x_idx = X
    # ts = np.arange(Nt)[:, None]
    # t_idx = np.broadcast_to(ts, Z.shape)

    for varname, var_data in var_data_dict.items():
        # VAR[varname] = var_data[t_idx, z_idx, y_idx, x_idx]
        VAR[varname] = var_data[z_idx, y_idx, x_idx]

    return [VAR[varname] for varname in varnames]
    # return VAR

In [ ]:
def GetVariables(inputDataDirectory,timeString, Z,Y,X):
    # print('Making W and QCQI Lagrangian Array')
    varnames_W=['wb_hadv','wb_vadv','wb_hidiff','wb_vidiff',
                'wb_hturb','wb_vturb','wb_pgrad','wb_buoy']
    varnames_QV=['qvb_hadv','qvb_vadv','qvb_hidiff','qvb_vidiff',
                 'qvb_hturb','qvb_vturb','qvb_mp']
    varnames_TH=['ptb_hadv','ptb_vadv','ptb_hidiff','ptb_vidiff',
                 'ptb_hturb','ptb_vturb','ptb_mp','ptb_rad','ptb_div','ptb_diss']

    [WB_HADV, WB_VADV, WB_HIDIFF, WB_VIDIFF,
     WB_HTURB, WB_VTURB, WB_PGRAD, WB_BUOY] = MakeLagrangianArray(
        DataManager, timeString, varnames_W, Z, Y, X
    )
    
    [QVB_HADV, QVB_VADV, QVB_HIDIFF, QVB_VIDIFF,
     QVB_HTURB, QVB_VTURB, QVB_MP] = MakeLagrangianArray(
        DataManager, timeString, varnames_QV, Z, Y, X
    )
    
    [PTB_HADV, PTB_VADV, PTB_HIDIFF, PTB_VIDIFF,
     PTB_HTURB, PTB_VTURB, PTB_MP, PTB_RAD, PTB_DIV, PTB_DISS] = MakeLagrangianArray(
        DataManager, timeString, varnames_TH, Z, Y, X
    )
    
    outputDictionary_W = {
        "WB_HADV": WB_HADV,
        "WB_VADV": WB_VADV,
        "WB_HIDIFF": WB_HIDIFF,
        "WB_VIDIFF": WB_VIDIFF,
        "WB_HTURB": WB_HTURB,
        "WB_VTURB": WB_VTURB,
        "WB_PGRAD": WB_PGRAD,
        "WB_BUOY": WB_BUOY,
    }
    
    outputDictionary_QV = {
        "QVB_HADV": QVB_HADV,
        "QVB_VADV": QVB_VADV,
        "QVB_HIDIFF": QVB_HIDIFF,
        "QVB_VIDIFF": QVB_VIDIFF,
        "QVB_HTURB": QVB_HTURB,
        "QVB_VTURB": QVB_VTURB,
        "QVB_MP": QVB_MP,
    }
    
    outputDictionary_TH = {
        "PTB_HADV": PTB_HADV,
        "PTB_VADV": PTB_VADV,
        "PTB_HIDIFF": PTB_HIDIFF,
        "PTB_VIDIFF": PTB_VIDIFF,
        "PTB_HTURB": PTB_HTURB,
        "PTB_VTURB": PTB_VTURB,
        "PTB_MP": PTB_MP,
        "PTB_RAD": PTB_RAD,
        "PTB_DIV": PTB_DIV,
        "PTB_DISS": PTB_DISS,
    }

    return outputDictionary_W,outputDictionary_QV,outputDictionary_TH 

In [ ]:
###############################################################
#RUNNING

In [ ]:
#CALCULATING AND APPENDING TO DATA EACH TIMESTEP
for t in tqdm(loop_elements, total=len(loop_elements)):
    if np.mod(t,1)==0: print(f'Current time {t}')

    #getting timestring for loading input data
    timeString = ModelData.timeStrings[t]

    #loading input variables
    z,x,Z,Y,X = GetZYX(DataManager.inputParcelDirectory, timeString)

    #calculating variables
    [outputDictionary_W,outputDictionary_QV,outputDictionary_TH] = GetVariables(DataManager,timeString, Z,Y,X)
    
    #outputting
    DataManager.SaveOutputTimestep(DataManager.outputDataDirectory, timeString, outputDictionary_W, dataName=DataManager.dataName+"_W")
    DataManager.SaveOutputTimestep(DataManager.outputDataDirectory, timeString, outputDictionary_QV, dataName=DataManager.dataName+"_QV")
    DataManager.SaveOutputTimestep(DataManager.outputDataDirectory, timeString, outputDictionary_TH, dataName=DataManager.dataName+"_TH")

In [ ]:
######################################################

In [ ]:
# #READING BACK IN
# def GetVariableData(t, dataType="BUDGET_VARS", dataName="BUDGET_VARS_W"):#BUDGET_VARS_W, BUDGET_VARS_QV, BUDGET_VARS_TH
#     res = ModelData.res
#     t_res = ModelData.t_res
#     Nz_str = ModelData.Nz_str
#     inputDirectory = os.path.join(DataManager.outputDirectory, f"{res}_{t_res}_{Nz_str}nz", dataType)
#     timeString = ModelData.timeStrings[t]

#     FileName = os.path.join(inputDirectory, f"{dataName}_{res}_{t_res}_{Nz_str}nz_{timeString}.h5")

#     dataDictionary = {}
#     with h5py.File(FileName, 'r') as f:
#         print("Keys in file:", list(f.keys()))
#         for key in f.keys():
#             dataDictionary[key] = f[key][:]
#             print(f"{key}: shape = {dataDictionary[key].shape}, dtype = {dataDictionary[key].dtype}")
#     return dataDictionary

# dataDictionary = GetVariableData(t=0)